In [1]:
import timeit
import math
from datetime import datetime

class Node :
    
    def __init__(self,nbObjects,curBin,nbBins, cRemaining,last): 
        
        self.nbObjects=nbObjects           # number of the first k objects packing  
        self.nbBins=nbBins                 # number of bins used to pack the first k objects
        self.cRemaining=cRemaining         # a table of remaining capacities of each one of nbBins used 
        self.curBin=curBin
        self.last=last

        
    def getNbObjects(self):
        return self.nbObjects  
    
    def getNbBins(self):
        return self.nbBins
    
    def getCurBin(self):
        return self.curBin
        
    def getIcRemaining(self,i):
        return self.cRemaining[i]
    
    def getCremaining(self):
        return self.cRemaining
    
    def getLast(self):
        return self.last  
    

    
    def printNode(self):
        print("objects",self.nbObjects)
        print("nbBins",self.nbBins)
        print("curBin",self.curBin)
        print("cRem",self.cRemaining)
        print("last",self.last)

    
def branchAndBound(n,objects,c):
    # n: number of objects which is the max number of bins we can use 
    # objects: table of weights of objects 
    # c: capacity of each bin
    objects.sort(reverse=True)
    time = timeit.default_timer()
    minBins= n # initialize upper bound (Sup)
    usedBins=0 # initialize the number of used Bins
    cRemaining= [c]*n # initialize the table of remaining cpacaties of each bin
    nodes=[] # array that will contain created nodes and not processed 
    curBin=0
    cumWeight=[]
    precWeight=0
    newCremaining=cRemaining.copy()
    last=False
    for i in range (n):
        cumWeight.append(precWeight+objects[i])
        precWeight=precWeight+objects[i]
        j=0
        while newCremaining[j]< objects[i]:
            j+=1
        
        if j==usedBins:
            last=True
            usedBins+=1
        else:
            last=False
        newCremaining[j]-=objects[i]
        nodes.append(Node(i,j,usedBins,newCremaining.copy(),last))    
    minBins=usedBins
    #print("min",minBins)
    #

    #
    while (len(nodes)>0):
            #
            node=nodes.pop(-1)
            nbObjects=node.getNbObjects()+1
            last=node.getLast()
            tRemaining=node.getCremaining().copy()
            usedBins=node.getNbBins()
            #print("use",usedBins)
            if(nbObjects==n):
                    if(usedBins<minBins):
                        minBins=usedBins
                    while(last  and len(nodes)>0):
                        node=nodes.pop(-1)
                        last=node.getLast()
                        #print(node.getNbBins())
                    curBin=node.getCurBin()+1
                    
                    node=nodes.pop(-1)
                    nbObjects=node.getNbObjects()+1
                    tRemaining=node.getCremaining().copy()
                    usedBins=node.getNbBins()
                        
            while(tRemaining[curBin]< objects[nbObjects]):
                curBin+=1
            if (curBin==usedBins):
                last=True
                usedBins+=1
            else:
                last=False
            oRemaining=tRemaining[:usedBins]
            oRemaining[curBin]=oRemaining[curBin]-objects[nbObjects] 
            large=max(oRemaining)
            j=0
            if(nbObjects+1<n and large>=objects[n-1]):
                j=1
            #print(oRemaining, usedBins)
            ev=math.ceil(usedBins+(cumWeight[n-1]-cumWeight[nbObjects]-j*(usedBins*c-cumWeight[nbObjects]))/c)
            while(ev>minBins and not last):
                        curBin+=1
                        while(tRemaining[curBin]< objects[nbObjects]):
                                curBin+=1
                        if (curBin==usedBins):
                                last=True
                                usedBins+=1
                        else:
                                
                                last=False
                        oRemaining=tRemaining[:usedBins]
                        oRemaining[curBin]=oRemaining[curBin]-objects[nbObjects] 
                        large=max(oRemaining)
                        j=0
                        if(nbObjects+1<n and large>=objects[n-1] ):
                            j=1
                        ev=math.ceil(usedBins+(cumWeight[n-1]-cumWeight[nbObjects]-j*(usedBins*c-cumWeight[nbObjects]))/c)
            
            if(ev<=minBins):
                nodes.append(node)
                tRemaining[curBin]=tRemaining[curBin]-objects[nbObjects]                          
                nodes.append(Node(nbObjects,curBin,usedBins,tRemaining.copy(),last))
                curBin=0
                nbObjects=nbObjects+1
            else:                
                while (last and len(nodes)>0):
                        node=nodes.pop(-1) 
                        last=node.getLast()
                curBin=node.getCurBin()+1

    time = timeit.default_timer()-time
    return minBins

def readFromFile(file):
    fichier = open(file, "r")
    w=[]
    i=0
    for line in fichier:
        if i==0:
            n=int(line)
            i=i+1  
        else:
            if i==1:
                c=int(line)
                i=i+1
            else: 
                if i>1:
                    w.append(int(line.rstrip('\n\r')))
                    i=i+1
    fichier.close()
    return n,c,w,i
                

#n,c,w,i=readFromFile("./Scholl_1/N1C1W1_R.txt")
#n,c,w,i=readFromFile("./Randomly_Generated/BPP_750_75_0.2_0.8_9.txt")
#n,c,w,i=readFromFile("./Randomly_Generated/BPP_50_50_0.2_0.8_9.txt")
#n,c,w,i=readFromFile("./Randomly_Generated/BPP_100_75_0.1_0.8_5.txt")
#print(n,c,w,i)
#branchAndBound(n,w,c)  
#branchAndBound(7,[4,9,12,17,18,19,20],40)
#branchAndBound(12,[8,12,20,21,28,34,35,40,41,50,70,80],100)
#branchAndBound(12,[80,70,50,41,40,35,34,28,21,20,12,8],100)
#branchAndBound(5,[41,40,35,34,28],100)
#branchAndBound(10,[8,10,12,20,21,22,28,34,35,40,41,50,70,71,72,75,76,80,81,90],100)  
#branchAndBound(14,[90,81,80,76,75,72,71,70,50,41,40,35,34,28,22,21,20,12,10,8],100) 
#branchAndBound(7,[20,19,18,17,12,9,4],40)

branchAndBound(5,[41,40,35,34,28],90) 
#branchAndBound(7,[20,19,18,17,12,9,4],40)
#branchAndBound(10,[80,70,50,41,40,35,34,28,21,20],100)
#branchAndBound(12,[90,81,79,76,60,53,52,40,27,23,12,5],100)
#branchAndBound(14,[81,80,79,76,61,58,52,47,32,28,25,18,11,3],100)
#branchAndBound(18,[90,81,80,76,75,72,71,70,50,41,40,35,34,28,22,21,20,12],100)
#branchAndBound(20,[87,83,79,67,54,52,49,45,43,42,34,30,28,25,22,19,16,9,7,2],100)





3